In [1]:
import json_lines
import pandas as pd
import numpy as np
import seaborn as sns
import re
import warnings
warnings.filterwarnings("ignore")
from IPython.core import display as ICD

# Introduction

The following notebook manipulates a large 150,000 entry JSONL data file. It does so by first breaking out dictionaries in the name and address fields fields; it converts the dictionary keys into new dataframe columns. It then calculates metrics for the data frame, such as % of occupied fields and unique values within columns. I began by first proving all functions and metrics on a subset dataframe with 100 entries. After successfully completing  a set of tasks on the small dataframe, I'll scale up to the original dataframe. 


In [2]:
# Load data into list, and then load to dataframe
with json_lines.open('../data/ida_wrangling_exercise_data.2017-02-13.jsonl.gz') as f:
    data = [item for item in f]
    df = pd.DataFrame(data)

### Data exploration 

In [3]:
df.head()

,address,dob,email,id,name,phone,record_date,ssn
0,"{'street': '86314 David Pass Apt. 211', 'city'...",1971-06-30,opark@hotmail.com,01d68a4c598a45559c06f4df0b3d82cb,"{'firstname': 'Cynthia', 'lastname': 'Dawson',...",624-869-4610,2006-07-08T09:02:13,xxx-xx-2412
1,"20722 Coleman Villages\nEast Rose, SC 71064-5894",1965-09-09,sperez@armstrong.com,876ff718291d4397bb1e0477ceee6ad9,"{'firstname': 'Tamara', 'lastname': 'Myers'}",1-594-462-7759,2009-03-28T20:22:57,xxx-xx-8025
2,"{'street': '6676 Young Square', 'city': 'New J...",1993-04-12,uortiz@gmail.com,81753097bf7e4e2085982f422bdb9cda,"{'firstname': 'Jamie', 'lastname': 'Alexander'}",472.218.5065x389,2016-08-30T20:31:39,xxx-xx-0568
3,"0932 Gomez Drives\nLeefort, MD 46879-3166",1977-04-14,palmerdiane@yahoo.com,2c2f7154b80f40ca80d08c5adc54ea45,"{'firstname': 'Angela', 'lastname': 'Garcia', ...",1-663-109-4460x1080,2001-02-15T18:50:35,xxx-xx-9825
4,"{'street': '158 Smith Vista', 'city': 'East Sh...",1970-03-19,nancymaxwell@gmail.com,4f5263f339694d068e17ee7fdbb852b8,"{'firstname': 'Jennifer', 'lastname': 'Rodrigu...",233-423-3823,2014-06-21T14:36:01,xxx-xx-9104


In [4]:
'Number of rows {} '.format(len(df))

'Number of rows 150000 '

This initial dataframe has 150,000 entries. Some of the fields (e.g. address and name) are recorded as dictionaries and strings. Let's start working with a subset (first 100 entries ) of the data frame. Truncating the dataframe will speed up initial code experimentation. 

### Operating on subset of data 

In [116]:
# Subset wtih first 100 entries of dataframe
df_sub = df[0:100].copy()

## 1. Make a list of all of the nested named fields that appear in any record. Concatenate nested field names using a period '.' to defind named fields for nested records. Present the list in alphabetical order.

In [117]:
def expand_dict(dframe, dict_names):
    """This function will take a dataframe and dictionary list as inputs. 
    It will convert dictionary keys into seperate columns within the same dataframe by invoking the apply method.
    It returns a dataframe with concatenated columns"""

    # Creates an empty dataframe
    df_nested = pd.DataFrame()

    # Iterate through list of dictionaries
    for item in dict_names:
        # Expand dictionary keys into columns
        df_nested = pd.concat([df_nested, dframe[item].apply(pd.Series).add_prefix(
            (item + '.'))], axis=1)
        # Drops extra generated column from apply method
        df_nested.drop(item + '.0', axis=1, inplace=True)
        # Add original nested column
        df_nested[item] = dframe[item]
        # Replace nested fields with NaN, otherwise leave string value
        df_nested[item] = df_nested[item].apply(lambda x: np.nan if type(x) == dict else x)

    return df_nested

In [118]:
#Invoke the function above 
df_sub = expand_dict(df_sub, ['address', 'name'])
df_sub

,address.city,address.state,address.street,address.zip,address,name.firstname,name.lastname,name.middlename,name
0,Hoodburgh,RI,86314 David Pass Apt. 211,83973,NaN,Cynthia,Dawson,Claire,NaN
1,NaN,NaN,NaN,NaN,"20722 Coleman Villages\nEast Rose, SC 71064-5894",Tamara,Myers,NaN,NaN
2,New Julie,UT,6676 Young Square,73125,NaN,Jamie,Alexander,NaN,NaN
3,NaN,NaN,NaN,NaN,"0932 Gomez Drives\nLeefort, MD 46879-3166",Angela,Garcia,Alexis,NaN
4,East Sharonstad,ME,158 Smith Vista,42483,NaN,Jennifer,Rodriguez,NaN,NaN
5,Thompsonview,AR,461 Knapp Unions,02398,NaN,Keith,Meadows,NaN,NaN
6,NaN,NaN,NaN,NaN,"9524 Danielle Burg Apt. 849\nNorth Vickistad, ...",Shane,Garcia,NaN,NaN
7,NaN,NaN,NaN,NaN,"407 Patrick Shoal\nWilliamsshire, VI 49526",Charles,Bell,Lee,NaN
8,Chrisview,CT,3697 Mills Estates Apt. 499,70997-3753,NaN,NaN,NaN,NaN,Morgan Morris
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bernard Wilkerson


In [8]:
# An alphabetical order list of the fields 
sorted(list(df_sub.columns))

['address',
 'address.city',
 'address.state',
 'address.street',
 'address.zip',
 'name',
 'name.firstname',
 'name.lastname',
 'name.middlename']

## 2. Answer the following questions for each field in your list from question 1.

- What percentage of the records contain the field?
- What are the five most common values of the field?

#### What percentage of the records contain the field?

In [9]:
def col_percent(dframe):
    '''A funciton to loop through columns and return the percentage of populated items.'''
    for item in dframe:
        print('{} = {:.1f}%'.format(item, 100 *
                                    dframe[item].count() / len(dframe)))

In [10]:
# Invoke the function above 
col_percent(df_sub)

address.city = 41.0%
address.state = 41.0%
address.street = 41.0%
address.zip = 41.0%
address = 52.0%
name.firstname = 66.0%
name.lastname = 66.0%
name.middlename = 26.0%
name = 34.0%


#### What are the five most common values of the field?

In [11]:
# Five most common values in a column
def common_values(df, num):
    '''Function takes a dataframe and int as input, returns int number of common values'''
    for item in df:
        # Prints common values as a dataframe
        ICD.display(pd.DataFrame(df[item].value_counts().head(num)) )

In [12]:
common_values(df_sub,5)

,address.city
North Deborahside,1
Nolanmouth,1
Courtneyport,1
Chrisview,1
New Kelly,1


,address.state
RI,4
UT,4
WY,3
VI,2
LA,2


,address.street
3971 Paul Plaza Apt. 365,1
047 Mcclure Crossroad Apt. 704,1
93406 Justin Spur,1
56833 Baker Branch,1
17685 Jason Lane Apt. 833,1


,address.zip
42483,1
04527,1
56062-1568,1
64603,1
21960,1


,address
"2409 Alicia Port\nEast Mckenzie, KY 48124",1
"PSC 9738, Box 3367\nAPO AE 85789",1
"090 Laura Islands\nLake Jennifer, OR 78668-8235",1
"1471 Nichols Hill Apt. 029\nClintonport, AL 76610",1
"732 Martin Island\nBrownmouth, GU 90564",1


,name.firstname
Robert,3
Kevin,2
Steven,2
Shane,2
Eric,2


,name.lastname
Davis,3
White,2
Gonzalez,2
Chapman,2
Rodriguez,2


,name.middlename
David,2
Todd,1
Angela,1
Jennifer,1
Claire,1


,name
Lori Rivera,1
Jeremy Stephens,1
Deborah Good,1
Marc Rogers,1
Paul Russo,1


## 3. How many distinct first names appear in this data set? Explain your procedure for identifying distinct first names.

Used the unique() method on the 'name.namefirst' column. With more time I would have parsed out the first names strings in the 'name' column. 

In [13]:
df_sub['name.firstname'].unique()

array(['Cynthia', 'Tamara', 'Jamie', 'Angela', 'Jennifer', 'Keith',
       'Shane', 'Charles', nan, 'Amanda', 'Adrian', 'Tiffany', 'Peter',
       'Steven', 'Lindsay', 'Sara', 'Amy', 'Christopher', 'Melissa',
       'Crystal', 'Mallory', 'Elizabeth', 'Richard', 'Jonathan', 'Nicole',
       'Michael', 'Robert', 'Ryan', 'Mary', 'Brooke', 'Daniel', 'Calvin',
       'Brandon', 'Laurie', 'Ashley', 'Brent', 'Kevin', 'Eric', 'Deborah',
       'Christine', 'Joseph', 'Donald', 'Jeff', 'Nathan', 'Martin', 'Lisa',
       'Ricky', 'Casey', 'Stacy', 'Matthew', 'Cassidy', 'Jenna', 'Mark',
       'Brad', 'Stephanie', 'John', 'Lauren', 'Veronica'], dtype=object)

In [14]:
len(df_sub['name.firstname'].unique())

58

## 4. How many distinct street names appear in this data set? Explain your procedure for identifying distinct street names.


In [237]:
def street_names(df):
    '''A function to return the distinct street names. 
    The function takes a dataframe column as input. It removes street, Apt., and Suite numbers.
    Returns an dataframe with top 10 values and prints the number of distinct street names'''
    # Drops and NaN Values
    df = df.dropna()
    # remove numbers, and split at Apt. or Suite instances, only take first part of split
    df = df.apply(lambda x: re.sub("\d+", "", re.split('Apt. | Suite', x)[0]))
    ICD.display(pd.DataFrame(df.value_counts().head(10)))
    return 'There are {} distinct street names'.format(len(df.unique()))

In [238]:
street_names( df_sub['address.street'] )

,address.street
Vargas Common,1
David Pass,1
Ward Lake,1
Mccarthy Hill,1
Osborne Village,1
Bender Common,1
Barnes Loaf,1
Kenneth Trafficway,1
Jason Lane,1
Solis Loop,1


'There are 41 distinct street names'

## 5. What are the 5 most common US area codes in the phone number field? Explain your approach to identify the US area codes in this data set.

## 6. What are the 5 most common zip codes

In [225]:
def zip_codes(df):
    '''A function to return the 5 most common zip codes. It takes a list of dataframe columns as input. 
    First it drops an NaN values, then it grabs the last element of the value, finally, if there's a dash the value is split there.'''
    df_out = pd.DataFrame()
    for item in df:
        # Drops and NaN Values
        df = item.dropna()
        # grab the last element in address and remove numbers after dash
        df = df.apply(lambda x: re.split('-',  x.split()[-1])[0])
        df_out = pd.concat([df_out,df])
    return df_out[0].value_counts().head(5)

In [224]:
zip_codes([  df_sub['address'],df_sub['address.zip'] ])

43425    1
78668    1
45814    1
77615    1
97347    1
Name: 0, dtype: int64